In [ ]:
import pandas as pd
import sqlite3

In [ ]:
#Conecto a la base de datos
connection = sqlite3.connect("./sql-murder-mystery.db")

In [ ]:
#Obtengo el cursor para hacer las queries
game_cursor = connection.cursor()

In [ ]:
#Creo la funcion para leer datos y pasar a un dataframe de pandas

def sql_query(query):
    game_cursor.execute(query)
    ans = game_cursor.fetchall()
    names = [description[0] for description in game_cursor.description]
    #Modifico la funcion para poder ver las columnas enteras 
    for row in ans:
        print([str(col) for col in row])  # Convierte todas las columnas a string para evitar truncamientos
    
    return pd.DataFrame(ans, columns=names)


In [ ]:
#query para ver los nombres de las tablas
query = ''' 
SELECT name 
FROM sqlite_master
where type = 'table'
'''
sql_query(query)


In [ ]:
#Estructura de 'crime_scene_report' table
query = '''
SELECT * 
FROM crime_scene_report;
'''
sql_query(query)

In [ ]:
# Actualizo la columna 'date' al formato europeo
game_cursor.execute("""
    UPDATE crime_scene_report
    SET date = SUBSTR(date, 7, 2) || '-' || SUBSTR(date, 5, 2) || '-' || SUBSTR(date, 1, 4)
""")

## Pistas
Sabemos que el crimen fue un asesinato ➡️ Filtrar por type: murder  
Ocurrio el 15 de enero de 2018 ➡️ Filtrar date   
Sucedio en SQL City ➡️ Filtrar en City: SQL City  

In [ ]:
#Filtro por type: murder, date: 15-01-2018, city =SQL City
query='''
SELECT *
FROM crime_scene_report
WHERE type = "murder"
	AND date = "15-01-2018"
	AND city = "SQL City";
'''
sql_query(query)

Luego tenemos que ver quienes son los testigos involucrados: dice que el primer testigo vivio por ultima vez en "Northwestern Dr", y el segundo en "Franklin Ave".

In [ ]:
query='''
SELECT *
FROM person;
'''
sql_query(query)

In [ ]:
#identifico el primer testigo, que vive en la ultima casa de "Northwestern Dr":

query='''
SELECT *
FROM person
WHERE address_street_name = "Northwestern Dr"
ORDER BY address_number DESC;
'''
sql_query(query)

In [ ]:
#de igual manera identifico el segundo testigo, de nombre Annabel y que vive en "Franklin Ave"
query='''
SELECT *
FROM person
WHERE name LIKE '%Annabel%'
AND address_street_name = "Franklin Ave";
'''
sql_query(query)

Con los testigos identificados, revisamos sus entrevistas...

In [ ]:
query='''
SELECT *
FROM interview
WHERE person_id IN ("14887", "16371");
'''
sql_query(query)

Entonces las pistas son:   
- Es un hombre
- Va al gimnasio *Get Fit Now Gym*
- Tiene membresia Gold que empieza en "48Z"
- Fue a entrenar el dia 9 de enero
- Conduce un coche con matrícula que contiene los caracteres "H42W"


In [ ]:
#De acuerdo a las pistas usamos la tabla de get_fit_now_check_in
query='''
SELECT *
FROM get_fit_now_check_in
WHERE membership_id LIKE '48Z%'
	AND check_in_date = "20180109";
'''
sql_query(query)

In [ ]:
#De la misma forma usamos el dato de la matricula en la tabla 'drivers_license'
query='''
SELECT *
FROM drivers_license
WHERE gender = "male"
	AND plate_number LIKE '%H42W%';
'''
sql_query(query)



In [ ]:
#Buscamos en la tabla person por el numero de license_id


query='''
SELECT *
FROM person
WHERE license_id IN ("423327", "664760");
'''
sql_query(query)

In [ ]:
#Buscamos en la tabla get_fit_now_member de acuerdo a person_id
query='''
SELECT *
FROM get_fit_now_member
WHERE person_id IN ("51739", "67318");
'''
sql_query(query)


Y nuestro asesino ha sido encontrado! 

In [ ]:
connection.commit()